In [1]:
import sys
import time
import numpy as np
import pickle
import importlib

In [2]:
from qiskit import Aer, IBMQ
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

In [4]:
max_size = 57
max_length = 56

In [5]:
with open("pkls/raw_hist_list.pkl", "rb") as f:
    raw_hist_list = pickle.load(f)
with open("pkls/results_meas_cal.pkl", "rb") as f:
    results_meas_cal = pickle.load(f)

In [6]:
import libs_qrem
importlib.reload(libs_qrem)
from libs_qrem import QREM_Filter_1

In [7]:
xz_delta_mitigator_list = []
for n in range(2, max_size + 1): # [2, ..., 57]
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 2], mit_pattern=mit_pattern)
    xz_delta_mitigator_list.append(QREM_Filter_1(n, meas_fitter.cal_matrices))
    if n % 10 == 0:
        print("size", n, "finished")
print("length of xz_delta_mitigator_list: ", len(xz_delta_mitigator_list))

size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
length of xz_delta_mitigator_list:  56


In [8]:
zx_delta_mitigator_list = []
for n in range(2, max_size + 1): # [2, ..., 57]
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 2], mit_pattern=mit_pattern)
    zx_delta_mitigator_list.append(QREM_Filter_1(n, meas_fitter.cal_matrices))
    if n % 10 == 0:
        print("size", n, "finished")
print("length of zx_delta_mitigator_list: ", len(zx_delta_mitigator_list))

size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
length of zx_delta_mitigator_list:  56


In [9]:
delta_hist_list = []
xz_delta_total_times = []
zx_delta_total_times = []
for i in range(max_length):
    print(i + 2, "qubits")
    
    t1 = time.perf_counter()
    delta_hist_list.append( xz_delta_mitigator_list[i].apply(raw_hist_list[2 * i]) )
    t2 = time.perf_counter()
    xz_delta_total_times.append(t2 - t1)
    print("xz", t2 - t1, "s")
    
    t3 = time.perf_counter()
    delta_hist_list.append( zx_delta_mitigator_list[i].apply(raw_hist_list[2 * i + 1]) )
    t4 = time.perf_counter()
    zx_delta_total_times.append(t4 - t3)
    print("zx", t4 - t3, "s")
    
    print()

2 qubits
xz 0.0001047350000007441 s
zx 3.445000000112941e-05 s

3 qubits
xz 4.3829999999189795e-05 s
accumulator is positive, we might even ignoring the necessal positive values.
zx 4.9867999997843526e-05 s

4 qubits
xz 0.00012767400000157636 s
zx 9.983799999702114e-05 s

5 qubits
xz 0.00011160900000106722 s
zx 0.00010501599999912514 s

6 qubits
xz 0.00022697800000059942 s
zx 0.00026341799999940463 s

7 qubits
xz 0.0005186730000019679 s
zx 0.0007071100000004549 s

8 qubits
xz 0.001284617999999682 s
zx 0.0012120759999980635 s

9 qubits
xz 0.0021452209999992533 s
zx 0.0026429260000000454 s

10 qubits
xz 0.0067650940000021365 s
zx 0.00503165800000005 s

11 qubits
xz 0.008107737999999642 s
zx 0.017286039999998337 s

12 qubits
xz 0.025701648999998383 s
zx 0.02539707600000085 s

13 qubits
xz 0.02917275899999794 s
zx 0.05947332399999894 s

14 qubits
xz 0.07715632599999722 s
zx 0.08226380600000027 s

15 qubits
xz 0.09599179099999944 s
zx 0.1633474029999995 s

16 qubits
xz 0.1920474989999974 s


In [10]:
with open("./pkls/delta_hist_list.pkl", "wb") as f:
    pickle.dump(delta_hist_list, f)

In [11]:
with open("./pkls/delta_total_times.pkl", "wb") as f:
    pickle.dump({"xz": xz_delta_total_times, "zx": zx_delta_total_times}, f)

In [12]:
xz_delta_times = []
zx_delta_times = []
for i in range(max_length):
    xz_delta_times.append( xz_delta_mitigator_list[i].times() )
    zx_delta_times.append( zx_delta_mitigator_list[i].times() )

In [13]:
with open("./pkls/delta_times.pkl", "wb") as f:
    pickle.dump({"xz": xz_delta_times, "zx": zx_delta_times}, f)

In [14]:
xz_delta_mitigators = []
for i in range(max_length):
    xz_delta_mitigators.append({"indices_to_keys_vector":xz_delta_mitigator_list[i].indices_to_keys_vector(),
                              # "mitigated_hist":xz_delta_mitigator_list[i].mitigated_hist(),
                              "one_norm":xz_delta_mitigator_list[i].one_norm(),
                              # "reduced_inv_A":xz_delta_mitigator_list[i].reduced_inv_A(),
                              "sum_of_x":xz_delta_mitigator_list[i].sum_of_x(),
                              "sum_of_x_hat":xz_delta_mitigator_list[i].sum_of_x_hat(),
                              "sum_of_x_tilde":xz_delta_mitigator_list[i].sum_of_x_tilde(),
                              # "times":xz_delta_mitigator_list[i].times(),
                              "x_s":xz_delta_mitigator_list[i].x_s(),
                              "x_hat":xz_delta_mitigator_list[i].x_hat(),
                              "x_tilde":xz_delta_mitigator_list[i].x_tilde(),
                              })
    print(i, "finished")

0 finished
1 finished
2 finished
3 finished
4 finished
5 finished
6 finished
7 finished
8 finished
9 finished
10 finished
11 finished
12 finished
13 finished
14 finished
15 finished
16 finished
17 finished
18 finished
19 finished
20 finished
21 finished
22 finished
23 finished
24 finished
25 finished
26 finished
27 finished
28 finished
29 finished
30 finished
31 finished
32 finished
33 finished
34 finished
35 finished
36 finished
37 finished
38 finished
39 finished
40 finished
41 finished
42 finished
43 finished
44 finished
45 finished
46 finished
47 finished
48 finished
49 finished
50 finished
51 finished
52 finished
53 finished
54 finished
55 finished


In [15]:
zx_delta_mitigators = []
for i in range(max_length):
    zx_delta_mitigators.append({"indices_to_keys_vector":zx_delta_mitigator_list[i].indices_to_keys_vector(),
                              # "mitigated_hist":zx_delta_mitigator_list[i].mitigated_hist(),
                              "one_norm":zx_delta_mitigator_list[i].one_norm(),
                              # "reduced_inv_A":zx_delta_mitigator_list[i].reduced_inv_A(),
                              "sum_of_x":zx_delta_mitigator_list[i].sum_of_x(),
                              "sum_of_x_hat":zx_delta_mitigator_list[i].sum_of_x_hat(),
                              "sum_of_x_tilde":zx_delta_mitigator_list[i].sum_of_x_tilde(),
                              # "times":zx_delta_mitigator_list[i].times(),
                              "x_s":zx_delta_mitigator_list[i].x_s(),
                              "x_hat":zx_delta_mitigator_list[i].x_hat(),
                              "x_tilde":zx_delta_mitigator_list[i].x_tilde(),
                              })
    print(i, "finished")

0 finished
1 finished
2 finished
3 finished
4 finished
5 finished
6 finished
7 finished
8 finished
9 finished
10 finished
11 finished
12 finished
13 finished
14 finished
15 finished
16 finished
17 finished
18 finished
19 finished
20 finished
21 finished
22 finished
23 finished
24 finished
25 finished
26 finished
27 finished
28 finished
29 finished
30 finished
31 finished
32 finished
33 finished
34 finished
35 finished
36 finished
37 finished
38 finished
39 finished
40 finished
41 finished
42 finished
43 finished
44 finished
45 finished
46 finished
47 finished
48 finished
49 finished
50 finished
51 finished
52 finished
53 finished
54 finished
55 finished


In [16]:
with open("./pkls/delta_mitigator_list.pkl", "wb") as f:
    pickle.dump({"xz": xz_delta_mitigators, "zx": zx_delta_mitigators}, f)